In [1]:
# Import dependencies
import requests
import pprint
import pandas as pd
import json
import os
import csv
from pymongo import MongoClient
from pprint import pprint

In [2]:
properties_original_df = pd.read_csv("Resources\historical_property-sales-data.csv")
properties_original_df.head()

C:\Users\Ajunjee Selvam\AppData\Local\Temp\ipykernel_24812\3404626824.py:1: DtypeWarning: Columns (12,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  properties_original_df = pd.read_csv("Resources\historical_property-sales-data.csv")


,PropertyID,PropType,taxkey,Address,CondoProject,District,nbhd,Style,Extwall,Stories,Year_Built,Rooms,FinishedSqft,Units,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price
0,98459.0,Commercial,30091000,9220 N 107TH ST,NaN,9.0,6202.0,Office Building - 1 Story,Concrete Block,1.0,1981.0,NaN,12960,5,NaN,0.0,0.0,54885,2023-08-15,530000
1,98477.0,Residential,39995000,9574 N 107TH ST,NaN,9.0,40.0,Cape Cod,Aluminum/Vinyl,1.0,1942.0,7.0,1182,1,4.0,1.0,1.0,33541,2023-12-15,160000
2,98500.0,Residential,40071000,9531 N MICHAEL CT,NaN,9.0,40.0,Ranch,Fiber Cement/Hardiplank,1.0,2006.0,9.0,1880,1,3.0,2.0,0.0,10607,2023-08-31,387500
3,98566.0,Residential,50047000,9350 N JOYCE AV,NaN,9.0,40.0,Ranch,Aluminum/Vinyl,1.0,1980.0,6.0,1489,1,3.0,3.0,0.0,8640,2023-08-25,335000
4,98598.0,Residential,50079000,9355 N BURBANK AV,NaN,9.0,40.0,Ranch,Aluminum/Vinyl,1.0,1986.0,5.0,1209,1,3.0,1.0,1.0,7200,2023-06-20,250000


In [3]:
properties_original_df.columns

Index(['PropertyID', 'PropType', 'taxkey', 'Address', 'CondoProject',
       'District', 'nbhd', 'Style', 'Extwall', 'Stories', 'Year_Built',
       'Rooms', 'FinishedSqft', 'Units', 'Bdrms', 'Fbath', 'Hbath', 'Lotsize',
       'Sale_date', 'Sale_price'],
      dtype='object')

In [4]:
properties_original_df.shape

(56232, 20)

In [5]:
properties_original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56232 entries, 0 to 56231
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PropertyID    32049 non-null  float64
 1   PropType      56224 non-null  object 
 2   taxkey        56232 non-null  int64  
 3   Address       56228 non-null  object 
 4   CondoProject  9164 non-null   object 
 5   District      56230 non-null  float64
 6   nbhd          56175 non-null  float64
 7   Style         56141 non-null  object 
 8   Extwall       45726 non-null  object 
 9   Stories       56063 non-null  float64
 10  Year_Built    56171 non-null  float64
 11  Rooms         54602 non-null  float64
 12  FinishedSqft  56131 non-null  object 
 13  Units         56232 non-null  int64  
 14  Bdrms         54603 non-null  float64
 15  Fbath         55577 non-null  float64
 16  Hbath         51392 non-null  float64
 17  Lotsize       56231 non-null  object 
 18  Sale_date     56232 non-nu

What about the Style - do we filter for that as well? Because there's a lot of different types included.

In [6]:
residential_properties_df = properties_original_df.loc[properties_original_df['PropType'] == "Residential"]
residential_properties_df = residential_properties_df.drop(columns=['taxkey', 'CondoProject', 'District','nbhd', 'Extwall', 'Stories','Units', 'Rooms'], axis=1)
residential_properties_df.isnull().sum()

PropertyID      18069
PropType            0
Address             3
Style              87
Year_Built         26
FinishedSqft       90
Bdrms             128
Fbath              10
Hbath            3610
Lotsize             0
Sale_date           0
Sale_price          8
dtype: int64

In [7]:
residential_properties_df['FinishedSqft'] = residential_properties_df['FinishedSqft'].apply(pd.to_numeric, errors='coerce')
residential_properties_df['Lotsize'] = residential_properties_df['Lotsize'].apply(pd.to_numeric, errors='coerce')
residential_properties_df['Sale_price'] = residential_properties_df['Sale_price'].apply(pd.to_numeric, errors='coerce')
residential_properties_df['Sale_date'] = pd.to_datetime(residential_properties_df['Sale_date'], infer_datetime_format=True, format='mixed')
residential_properties_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43742 entries, 1 to 56182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   PropertyID    25673 non-null  float64       
 1   PropType      43742 non-null  object        
 2   Address       43739 non-null  object        
 3   Style         43655 non-null  object        
 4   Year_Built    43716 non-null  float64       
 5   FinishedSqft  34797 non-null  float64       
 6   Bdrms         43614 non-null  float64       
 7   Fbath         43732 non-null  float64       
 8   Hbath         40132 non-null  float64       
 9   Lotsize       33264 non-null  float64       
 10  Sale_date     43742 non-null  datetime64[ns]
 11  Sale_price    33228 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 4.3+ MB


C:\Users\Ajunjee Selvam\AppData\Local\Temp\ipykernel_24812\955635428.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  residential_properties_df['Sale_date'] = pd.to_datetime(residential_properties_df['Sale_date'], infer_datetime_format=True, format='mixed')


In [8]:
residential_properties_df = residential_properties_df[residential_properties_df['PropType'].notna()]
residential_properties_df = residential_properties_df[residential_properties_df['Bdrms'].notna()]
residential_properties_df = residential_properties_df[residential_properties_df['Fbath'].notna()]
residential_properties_df = residential_properties_df[residential_properties_df['Sale_price'].notna()]
residential_properties_df['Hbath'].fillna(0,inplace=True)
residential_properties_df.isnull().sum()

PropertyID      18069
PropType            0
Address             0
Style               0
Year_Built          0
FinishedSqft        0
Bdrms               0
Fbath               0
Hbath               0
Lotsize             0
Sale_date           0
Sale_price          0
dtype: int64

In [9]:
residential_properties_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33153 entries, 1 to 56182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   PropertyID    15084 non-null  float64       
 1   PropType      33153 non-null  object        
 2   Address       33153 non-null  object        
 3   Style         33153 non-null  object        
 4   Year_Built    33153 non-null  float64       
 5   FinishedSqft  33153 non-null  float64       
 6   Bdrms         33153 non-null  float64       
 7   Fbath         33153 non-null  float64       
 8   Hbath         33153 non-null  float64       
 9   Lotsize       33153 non-null  float64       
 10  Sale_date     33153 non-null  datetime64[ns]
 11  Sale_price    33153 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 3.3+ MB


In [10]:
residential_properties_df.isnull().sum()

PropertyID      18069
PropType            0
Address             0
Style               0
Year_Built          0
FinishedSqft        0
Bdrms               0
Fbath               0
Hbath               0
Lotsize             0
Sale_date           0
Sale_price          0
dtype: int64

In [11]:
residential_properties_df['year'] = residential_properties_df['Sale_date'].dt.year
residential_properties_df['month'] = residential_properties_df['Sale_date'].dt.month
residential_properties_df.head()

,PropertyID,PropType,Address,Style,Year_Built,FinishedSqft,Bdrms,Fbath,Hbath,Lotsize,Sale_date,Sale_price,year,month
1,98477.0,Residential,9574 N 107TH ST,Cape Cod,1942.0,1182.0,4.0,1.0,1.0,33541.0,2023-12-15,160000.0,2023,12
2,98500.0,Residential,9531 N MICHAEL CT,Ranch,2006.0,1880.0,3.0,2.0,0.0,10607.0,2023-08-31,387500.0,2023,8
3,98566.0,Residential,9350 N JOYCE AV,Ranch,1980.0,1489.0,3.0,3.0,0.0,8640.0,2023-08-25,335000.0,2023,8
4,98598.0,Residential,9355 N BURBANK AV,Ranch,1986.0,1209.0,3.0,1.0,1.0,7200.0,2023-06-20,250000.0,2023,6
5,98624.0,Residential,8210 W GLENBROOK RD,Colonial,1989.0,1663.0,3.0,2.0,1.0,7200.0,2023-01-10,270000.0,2023,1


In [12]:
residential_properties_df.to_csv("Resources\cleaned-property-sales-data.csv")